In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
import category_encoders
from catboost import Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost

In [2]:
train_data = pd.read_csv('./train_dataset/train_public.csv')
test_public = pd.read_csv('./test_public.csv')
train_internet = pd.read_csv('./train_dataset/train_internet.csv')

In [3]:
X_train = train_data.copy()
X_test = test_public.copy()
X_internet = train_internet.sample(n=20000)

In [4]:
X_train = X_train.drop(['isDefault'], axis = 1)
X_train.shape

(10000, 38)

In [5]:
X_train['is_train'] = 1
X_test['is_train'] = 0

X_traintest = X_traintest = pd.concat([X_train, X_test], axis = 0)
X_traintest.shape

(15000, 39)

In [6]:
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}



X_traintest['num_work_year'] = X_traintest['work_year'].map(work_year_dict)

X_internet['num_work_year'] = X_internet['work_year'].map(work_year_dict)

In [7]:
class_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
}



X_traintest['num_class'] = X_traintest['class'].map(class_dict)

X_internet['num_class'] = X_internet['class'].map(class_dict)

In [8]:
X_traintest['issue_date'] = pd.to_datetime(X_traintest['issue_date'])
X_internet['issue_date'] = pd.to_datetime(X_internet['issue_date'])


X_traintest['issue_date_delta'] = (X_traintest['issue_date'] - X_traintest['issue_date'].min())  / np.timedelta64(1,'D')
X_internet['issue_date_delta'] = (X_internet['issue_date'] - X_internet['issue_date'].min())  / np.timedelta64(1,'D')


In [9]:
X_traintest['earlies_credit_mon'] = pd.to_datetime(X_traintest['earlies_credit_mon'])
X_internet['earlies_credit_mon'] = pd.to_datetime(X_internet['earlies_credit_mon'])

X_traintest['earlies_credit_mon_delta'] = (X_traintest['earlies_credit_mon'] - X_traintest['earlies_credit_mon'].min())  / np.timedelta64(1,'D')
X_internet['earlies_credit_mon_delta'] = (X_internet['earlies_credit_mon'] - X_internet['earlies_credit_mon'].min())  / np.timedelta64(1,'D')

In [10]:
from sklearn.preprocessing import LabelEncoder

lbl1 = LabelEncoder().fit(X_internet['work_type'])

X_internet['work_type'] = lbl1.transform(X_internet['work_type'])


y_train1 = X_internet['house_loan_status']
y_train2 = X_internet['marriage']
y_train3 = X_internet['offsprings']
y_train4 = X_internet['work_type']

In [11]:
features_public = X_traintest.columns
features_internet = X_internet.columns

diff1 = features_public.difference(features_internet)
diff2 = features_internet.difference(features_public)

In [12]:
print(diff1)
print(diff2)

Index(['app_type', 'is_train', 'known_dero', 'known_outstanding_loan'], dtype='object')
Index(['f5', 'house_loan_status', 'is_default', 'marriage', 'offsprings',
       'sub_class', 'work_type'],
      dtype='object')


In [13]:
tmp_internet = X_internet.copy()
for cat in diff2:
    tmp_internet = tmp_internet.drop(cat, axis=1)
    
tmp_public = X_traintest.copy()
for cat in diff1:
    tmp_public = tmp_public.drop(cat, axis=1)

In [14]:
cat_drop = ['region', 'early_return', 'employer_type', 'industry', 'issue_date', 'earlies_credit_mon']
for cat in cat_drop:
    tmp_internet = tmp_internet.drop(cat, axis=1)
    tmp_public = tmp_public.drop(cat, axis=1)

In [15]:
numerical_fea_internet = list(tmp_public.select_dtypes(exclude=['object']).columns) 
category_fea_internet = list(filter(lambda x: x not in numerical_fea_internet,list(tmp_public.columns)))

In [16]:
tmp_internet[category_fea_internet] =tmp_internet[category_fea_internet].astype(str)

tmp_internet[numerical_fea_internet] = tmp_internet[numerical_fea_internet].astype(float)

tmp_public[category_fea_internet] = tmp_public[category_fea_internet].astype(str)

tmp_public[numerical_fea_internet] = tmp_public[numerical_fea_internet].astype(float)

In [17]:
clf_multiclass1 = catboost.CatBoostClassifier(
    iterations=20000,
    od_type='Iter',
    od_wait=120,
    max_depth=8,
    learning_rate=0.02,
    l2_leaf_reg=9,
    random_seed=2019,
    metric_period=500,
    fold_len_multiplier=1.1,
    loss_function='MultiClass',
    logging_level='Verbose',
    task_type="GPU"
    )

clf_multiclass1.fit(tmp_internet, y_train1, cat_features = category_fea_internet)

0:	learn: 1.0941558	total: 29.2ms	remaining: 9m 43s
500:	learn: 0.8929563	total: 12.7s	remaining: 8m 16s
1000:	learn: 0.8404973	total: 25.4s	remaining: 8m 1s
1500:	learn: 0.7940831	total: 38s	remaining: 7m 48s
2000:	learn: 0.7517768	total: 50s	remaining: 7m 29s
2500:	learn: 0.7107660	total: 1m 1s	remaining: 7m 12s
3000:	learn: 0.6723454	total: 1m 13s	remaining: 6m 56s
3500:	learn: 0.6355363	total: 1m 25s	remaining: 6m 43s
4000:	learn: 0.6013025	total: 1m 37s	remaining: 6m 30s
4500:	learn: 0.5706458	total: 1m 49s	remaining: 6m 15s
5000:	learn: 0.5409146	total: 2m 1s	remaining: 6m 3s
5500:	learn: 0.5115911	total: 2m 14s	remaining: 5m 53s
6000:	learn: 0.4841714	total: 2m 26s	remaining: 5m 40s
6500:	learn: 0.4583487	total: 2m 37s	remaining: 5m 27s
7000:	learn: 0.4339446	total: 2m 49s	remaining: 5m 15s
7500:	learn: 0.4116954	total: 3m 1s	remaining: 5m 2s
8000:	learn: 0.3901182	total: 3m 13s	remaining: 4m 49s
8500:	learn: 0.3699325	total: 3m 25s	remaining: 4m 37s
9000:	learn: 0.3513555	total

In [18]:
clf_multiclass2 = catboost.CatBoostClassifier(
    iterations=20000,
    od_type='Iter',
    od_wait=120,
    max_depth=8,
    learning_rate=0.02,
    l2_leaf_reg=9,
    random_seed=2019,
    metric_period=500,
    fold_len_multiplier=1.1,
    loss_function='MultiClass',
    logging_level='Verbose',
    task_type="GPU"
    )

clf_multiclass2.fit(tmp_internet, y_train2, cat_features = category_fea_internet)

0:	learn: 1.3720708	total: 33.1ms	remaining: 11m 1s
500:	learn: 0.7940034	total: 13.6s	remaining: 8m 50s
1000:	learn: 0.7394705	total: 26.9s	remaining: 8m 29s
1500:	learn: 0.6918580	total: 40s	remaining: 8m 12s
2000:	learn: 0.6501011	total: 53.2s	remaining: 7m 58s
2500:	learn: 0.6113561	total: 1m 6s	remaining: 7m 45s
3000:	learn: 0.5767244	total: 1m 19s	remaining: 7m 32s
3500:	learn: 0.5449128	total: 1m 33s	remaining: 7m 19s
4000:	learn: 0.5144488	total: 1m 46s	remaining: 7m 6s
4500:	learn: 0.4838046	total: 2m	remaining: 6m 53s
5000:	learn: 0.4561188	total: 2m 13s	remaining: 6m 41s
5500:	learn: 0.4299720	total: 2m 27s	remaining: 6m 28s
6000:	learn: 0.4056550	total: 2m 40s	remaining: 6m 14s
6500:	learn: 0.3836453	total: 2m 54s	remaining: 6m 1s
7000:	learn: 0.3634653	total: 3m 7s	remaining: 5m 48s
7500:	learn: 0.3453165	total: 3m 20s	remaining: 5m 34s
8000:	learn: 0.3274503	total: 3m 34s	remaining: 5m 20s
8500:	learn: 0.3109540	total: 3m 47s	remaining: 5m 7s
9000:	learn: 0.2955932	total:

In [19]:
clf_multiclass3 = catboost.CatBoostClassifier(
    iterations=20000,
    od_type='Iter',
    od_wait=120,
    max_depth=8,
    learning_rate=0.02,
    l2_leaf_reg=9,
    random_seed=2019,
    metric_period=500,
    fold_len_multiplier=1.1,
    loss_function='MultiClass',
    logging_level='Verbose',
    task_type="GPU"
    )

clf_multiclass3.fit(tmp_internet, y_train3, cat_features = category_fea_internet)


0:	learn: 1.7637805	total: 33.5ms	remaining: 11m 10s
500:	learn: 1.1041164	total: 16s	remaining: 10m 24s
1000:	learn: 1.0383784	total: 31.1s	remaining: 9m 50s
1500:	learn: 0.9797992	total: 45.9s	remaining: 9m 26s
2000:	learn: 0.9241198	total: 1m	remaining: 9m 8s
2500:	learn: 0.8680066	total: 1m 16s	remaining: 8m 52s
3000:	learn: 0.8154612	total: 1m 31s	remaining: 8m 40s
3500:	learn: 0.7653034	total: 1m 47s	remaining: 8m 28s
4000:	learn: 0.7169089	total: 2m 3s	remaining: 8m 15s
4500:	learn: 0.6714661	total: 2m 19s	remaining: 8m 1s
5000:	learn: 0.6279961	total: 2m 35s	remaining: 7m 47s
5500:	learn: 0.5865367	total: 2m 51s	remaining: 7m 32s
6000:	learn: 0.5479124	total: 3m 7s	remaining: 7m 17s
6500:	learn: 0.5117346	total: 3m 23s	remaining: 7m 2s
7000:	learn: 0.4778204	total: 3m 39s	remaining: 6m 47s
7500:	learn: 0.4458759	total: 3m 55s	remaining: 6m 32s
8000:	learn: 0.4159400	total: 4m 11s	remaining: 6m 16s
8500:	learn: 0.3883381	total: 4m 28s	remaining: 6m 3s
9000:	learn: 0.3622001	tota

In [20]:
clf_multiclass4 = catboost.CatBoostClassifier(
    iterations=20000,
    od_type='Iter',
    od_wait=120,
    max_depth=8,
    learning_rate=0.02,
    l2_leaf_reg=9,
    random_seed=2019,
    metric_period=500,
    fold_len_multiplier=1.1,
    loss_function='MultiClass',
    logging_level='Verbose',
    task_type="GPU"
    )

clf_multiclass4.fit(tmp_internet, y_train4, cat_features = category_fea_internet)

0:	learn: 1.3782322	total: 29.4ms	remaining: 9m 48s
500:	learn: 1.0982908	total: 13.1s	remaining: 8m 31s
1000:	learn: 1.0360117	total: 26.5s	remaining: 8m 22s
1500:	learn: 0.9804488	total: 39.3s	remaining: 8m 3s
2000:	learn: 0.9258880	total: 51.5s	remaining: 7m 42s
2500:	learn: 0.8770386	total: 1m 3s	remaining: 7m 25s
3000:	learn: 0.8300067	total: 1m 15s	remaining: 7m 8s
3500:	learn: 0.7850735	total: 1m 27s	remaining: 6m 51s
4000:	learn: 0.7419185	total: 1m 39s	remaining: 6m 37s
4500:	learn: 0.6988025	total: 1m 51s	remaining: 6m 25s
5000:	learn: 0.6586601	total: 2m 4s	remaining: 6m 12s
5500:	learn: 0.6198199	total: 2m 17s	remaining: 6m 1s
6000:	learn: 0.5852550	total: 2m 30s	remaining: 5m 51s
6500:	learn: 0.5533406	total: 2m 43s	remaining: 5m 39s
7000:	learn: 0.5227770	total: 2m 55s	remaining: 5m 26s
7500:	learn: 0.4936158	total: 3m 7s	remaining: 5m 13s
8000:	learn: 0.4669235	total: 3m 20s	remaining: 5m
8500:	learn: 0.4414753	total: 3m 33s	remaining: 4m 48s
9000:	learn: 0.4176346	total

In [21]:
y_pred1 = clf_multiclass1.predict(tmp_public)
y_pred2 = clf_multiclass2.predict(tmp_public)
y_pred3 = clf_multiclass3.predict(tmp_public)
y_pred4 = clf_multiclass4.predict(tmp_public)

y_pred1=pd.DataFrame(y_pred1)
print(y_pred1.value_counts())

y_pred2=pd.DataFrame(y_pred2)
print(y_pred2.value_counts())

y_pred3=pd.DataFrame(y_pred3)
print(y_pred3.value_counts())

y_pred4=pd.DataFrame(y_pred4)
print(y_pred4.value_counts())

0    11103
1     3760
2      137
dtype: int64
1    8653
0    6347
dtype: int64
0    14895
2       45
3       35
5       17
1        5
4        3
dtype: int64
1    12806
0     1131
3      966
2       97
dtype: int64


In [22]:
new_features = pd.concat([y_pred1, y_pred2, y_pred3, y_pred4], axis=1)
new_features.columns=['house_loan_status', 'marriage', 'offsprings', 'work_type']

In [23]:
new_features.to_csv('./train_dataset/new_features.csv', index=0)